# Generate release notes

This notebook guides you through the process of creating release notes. Using `generate_release_objects.py` and the OpenAI API, we are able to automate the release notes authoring process. 

## Import necessary libraries

This cell imports any dependencies and some functions from `generate_release_objects.py`.

In [1]:
import requests
import subprocess
import json
import re
import shutil
import numpy as np
import datetime
import openai
from dotenv import load_dotenv
import os

from generate_release_objects import ReleaseURL, PR
from generate_release_objects import get_release_date, write_prs_to_file

## Set up OpenAI API

Running this cell grabs your OpenAI API secret key from your `.env` file. 

In [2]:
def setup_openai_api():
    """Loads .env file and updates the OpenAI API key. 
    
    Replace '../.env' with the relative path to your .env file.

    Modifies:
        openai.api_key
    """
    # Load environment variables
    load_dotenv('../.env') # replace to match your correct path

    # Get the OpenAI API key
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        raise EnvironmentError("OpenAI API key is not set in .env file.")

    # Set the API key for the OpenAI library
    openai.api_key = api_key

setup_openai_api()

## Set labels

This cell creates the main sections of the release notes. `label_hierarchy` shows the order in which updates will be shown.

In [3]:

label_to_category = {
    "highlight": "## Release highlights",
    "enhancement": "## Enhancements",
    "deprecation": "## Deprecations",
    "bug": "## Bug fixes",
    "documentation": "## Documentation"
}

categories = { 
    "highlight": [],
    "enhancement": [],
    "deprecation": [],
    "bug": [],
    "documentation": []
}

label_hierarchy = ["highlight", "deprecation", "bug", "enhancement", "documentation"]

## Collect GitHub URLs

Running this cell will prompt you to enter your GitHub release URLs. Keep pasting them in until you're done, then press enter again.
Example release URL: https://github.com/validmind/documentation/releases/tag/v2.4.4

In [4]:
def collect_github_urls(): 
    """Collects release URLs from user.

    Returns:
        List[ReleaseURL]: A list of ReleaseURL objects

    Exits:
        If the user presses enter and no URLs were entered
    """
    urls = []
    while True:
        url = input("Enter a full GitHub release URL (leave empty to finish): ")
        if not url:
            if not urls:  # Check if no URLs have been added yet
                print("Error: You must specify at least one full GitHub release URL.")
                exit(1)  # Exit the script with an error code
            break
        urls.append(ReleaseURL(url))
    return urls 

github_urls = collect_github_urls() # the only big global variable

## Set the release date
Running this cell will prompt you to enter the desired release date. 
The default is 3 business days from today if you enter nothing on this prompt.

In [5]:

release_datetime = get_release_date()
formatted_release_date = release_datetime.strftime("%Y-%b-%d").lower()
original_release_date = release_datetime.strftime("%B %-d, %Y")

## Create release folder

These lines will create a folder inside of `~/site/releases` for the release notes. The folder name is the release date, as per our convention.

In [6]:

directory_path = f"../site/releases/{formatted_release_date}/"
os.makedirs(directory_path, exist_ok=True)
output_file = f"{directory_path}release-notes.qmd"

## Start writing to release notes file
This block writes the title of the release notes into the final release notes file.

In [7]:

print("Generating & editing release notes ...")

with open(output_file, "w") as file:
    file.write(f"---\ntitle: \"{original_release_date}\"\n---\n\n")


Generating & editing release notes ...


## Set up release notes components
`release_components` will contain all the components of the release notes in the form of a dictionary. Later, we will merge everything together to form the release notes.

In [8]:
release_components = dict()
release_components.update(categories)

## Set the repository and tag name
This block checks every URL and assigns its repo name, such as `documentation` or `backend`, and its tag name.

In [9]:
for url in github_urls:
    url.set_repo_and_tag_name() 

## Extract PRs from each URL
This block finds all the pull requests from each URL and stores them somewhere safe.

In [10]:
ansi_escape = re.compile(r'\x1B\[[0-?]*[ -/]*[@-~]')

def notebook_extract_prs(self):
    """Extracts PRs from the release URL.

    Modifies:
        self.prs
        self.data_json
    """
    cmd_release = ['gh', 'api', f'repos/{self.repo_name}/releases/tags/{self.tag_name}']
    result_release = subprocess.run(cmd_release, capture_output=True, text=True)
    output_release = result_release.stdout.strip()

    output_release_clean = ansi_escape.sub('', output_release)

    try:
        self.data_json = json.loads(output_release_clean)
    except json.JSONDecodeError:
        print(f"Error: Unable to parse release data for URL '{self.url}'.")      
    
    if 'body' in self.data_json:
        body = self.data_json['body']
        pr_numbers = re.findall(r"https://github\.com/.+/pull/(\d+)", body)

        for pr_number in pr_numbers: # initialize PR objects using pr_numbers and add to list of PRs
            curr_PR = PR(self.repo_name, pr_number)
            self.prs.append(curr_PR)

    else:
        print(f"Error: No body found in release data for URL '{self.url}'.")

ReleaseURL.extract_prs = notebook_extract_prs

for url in github_urls:
    url.extract_prs() # initializes PR objects into a list for each URL

## Load PR data

Using the JSON data from the PRs, this block extracts and stores data from each PR.

In [11]:
def notebook_load_data_json(self):
        """Loads the JSON data from a PR to self.data_json, sets to None if any labels are 'internal'

        Modifies:
            self.data_json
        """
        cmd = ['gh', 'pr', 'view', self.pr_number, '--json', 'title,body,url,labels', '--repo', self.repo_name]
        result = subprocess.run(cmd, capture_output=True, text=True)
        output = result.stdout.strip()

        output_clean = ansi_escape.sub('', output)

        try:
            self.data_json = json.loads(output_clean)
        except json.JSONDecodeError:
            print(f"Error: Unable to parse PR data for PR number {self.pr_number} in repository {self.repo_name}.")
            return None
        
        if any(label['name'] == 'internal' for label in self.data_json['labels']):
            self.data_json = None  # Ignore PRs with the 'internal' label

PR.load_data_json = notebook_load_data_json

for url in github_urls:
    for pr in url.prs:
        pr.load_data_json() # loads json file into object


## Compile and edit release notes body

(20s)
Using the prompt below, this block feeds the body of each PR to ChatGPT for editing. If you find that the output is not quite right, edit the prompt and play around with it.

In [12]:
editing_instructions_body = """
    Please edit the provided technical content according to the following guidelines:

    - Use simple and neutral language in the active voice.
    - Address users directly in the second person with "you".
    - Use present tense by avoiding the use of "will".
    - Apply sentence-style capitalization to text
    - Always capitalize the first letter of text on each line.
    - Rewrite sentences that are longer than 25 words as multiple sentences.
    - Only split text across multiple lines if the text contains more than three sentences.
    - Avoid handwaving references to "it" or "this" by including the text referred to. 
    - Treat short text of less than ten words without a period at the end as a heading. 
    - Enclose any words joined by underscores in backticks (`) if they aren't already.
    - Remove exclamation marks from text.
    - Remove quotes around non-code words.
    - Remove the text "feat:" from the output
    - Maintain existing punctuation at the end of sentences.
    - Maintain all original hyperlinks for reference.
    - Preserve all comments in the format <!--- COMMENT ---> as they appear in the text.
    """

for url in github_urls:
    for pr in url.prs:
        if pr.data_json: 
            if pr.extract_external_release_notes(): pr.edit_text_with_openai(False, editing_instructions_body)


## Edit each title
This block does the same as above for the titles of each PR. The output below will show the original PR title, the title after some algorithmic changes, and the title after ChatGPT edits it. If you find that it's not good after editing with ChatGPT, feel free to edit the prompt below.

In [13]:
editing_instructions_title = """
    Please edit the provided technical content according to the following guidelines:

    - Use simple and neutral language in the active voice.
    - Address users directly in the second person with "you".
    - Use present tense by avoiding the use of "will".
    - Apply sentence-style capitalization to text
    - Always capitalize the first letter of text on each line.
    - Rewrite sentences that are longer than 25 words as multiple sentences.
    - Only split text across multiple lines if the text contains more than three sentences.
    - Avoid handwaving references to "it" or "this" by including the text referred to. 
    - Treat short text of less than ten words without a period at the end as a heading. 
    - Enclose any words joined by underscores in backticks (`) if they aren't already.
    - Remove exclamation marks from text.
    - Remove quotes around non-code words.
    - Remove the text "feat:" from the output
    - Maintain existing punctuation at the end of sentences.
    - Maintain all original hyperlinks for reference.
    - Preserve all comments in the format <!--- COMMENT ---> as they appear in the text.
    """

for url in github_urls:
    for pr in url.prs:
        if pr.data_json: 
            pr.title = pr.data_json['title']
            pr.clean_title(editing_instructions_title)


ValidMind Style Guide
ValidMind Style Guide
Valid mind style guide
First draft for sandbox instructions
First draft for sandbox instructions
First draft for sandbox instructions
Updated the About section to break down the articles into new categor…
Updated the About section to break down the articles into new categor…
Updated the about section to break down the articles into new categories
Quickstart docs site improvements
Quickstart docs site improvements
Quickstart docs site improvements
Add QuickStart video to the docs site
Add QuickStart video to the docs site
Add quickstart video to the docs site


## Set labels for each PR
This block takes the label data from each PR and assigns it to the PR.

In [14]:

for url in github_urls:
    for pr in url.prs:
        if pr.data_json: pr.labels = [label['name'] for label in pr.data_json['labels']]


## Assign PR details to PR
This block compiles all the data we found earlier for each PR into one place. 

In [15]:

for url in github_urls:
    for pr in url.prs:
        if pr.data_json: pr.pr_details = {
            'pr_number': pr.pr_number,
            'title': pr.cleaned_title,
            'full_title': pr.data_json['title'],
            'url': pr.data_json['url'],
            'labels': ", ".join(pr.labels),
            'notes': pr.edited_text
        }


## Combine all PR data into the same release notes components
Now, we can take all the details we compiled above and append them to our final release notes components. Since we want to show features in order of importance, we sort by the priority of the label.

In [16]:

for url in github_urls:
    for pr in url.prs:
        if pr.data_json:
            assigned = False 
            for priority_label in label_hierarchy:
                if priority_label in pr.labels:
                    release_components[priority_label].append(pr.pr_details)
                    assigned = True
                    break
            if not assigned:
                release_components.setdefault('other', []).append(pr.pr_details)

## Write release notes to file
Now that `release_components` contains everything we need for the release notes, we can write it to our release notes file.

In [17]:
# Write categorized PRs to the file
with open(output_file, "a") as file:
    write_prs_to_file(file, release_components, label_to_category)


## Update sidebar
This block will go into our `_quarto.yml` file and add the new release notes so it shows up on the sidebar of the docsite. 

In [18]:

def update_quarto_yaml(release_date):
    """Updates the _quarto.yml file to include the release notes file so it can be accessed on the website.

    Params:
        release_date - release notes use the release date as the file name.
    
    Modifies:
        _quarto.yml file
    """
    yaml_filename = "../site/_quarto.yml"
    temp_yaml_filename = "../site/_quarto_temp.yml"

    # Copy the original YAML file to a temporary file
    shutil.copyfile(yaml_filename, temp_yaml_filename)

    with open(temp_yaml_filename, 'r') as file:
        lines = file.readlines()

    # Format the release date for insertion into the YAML file
    formatted_release_date = release_date.strftime("%Y-%b-%d").lower()

    with open(yaml_filename, 'w') as file:
        add_release_content = False
        insert_index = -1

        for i, line in enumerate(lines):
            file.write(line)
            if line.strip() == "# MAKE-RELEASE-NOTES-EMBED-MARKER":
                add_release_content = True
                insert_index = i

            if add_release_content and i == insert_index:
                file.write(f'        - releases/{formatted_release_date}/release-notes.qmd\n')
                add_release_content = False

    # Remove the temporary file
    os.remove(temp_yaml_filename)
    
    print(f"Added release notes to _quarto.yml, line {insert_index + 2}")

update_quarto_yaml(release_datetime)

Added release notes to _quarto.yml, line 106


## Show files to commit

In [19]:
# After completing all tasks, print git status to show output files
try:
    result = subprocess.run(["git", "status", "--short"], check=True, text=True, capture_output=True)
    lines = result.stdout.split('\n')
    print("Files to commit:")
    for line in lines:
        if line.startswith((' M', '??', 'A ')):
            print(line)
except subprocess.CalledProcessError as e:
    print("Failed to run git status:", e)

Files to commit:
 M generate-release-notes1.ipynb
 M ../site/_quarto.yml
?? ../site/releases/2024-jul-31/


## Preview and edit changes
Run this cell to preview your changes, and make edits to the release notes file you just generated.

In [21]:
%%bash
cd ../site
quarto preview

Preparing to preview
[1/1] releases/2024-jul-31/release-notes.qmd
WARN: Unable to resolve link target: releases/about/overview.qmd
WARN: Unable to resolve link target: releases/about/style-guide.qmd
WARN: Unable to resolve link target: releases/about/overview.qmd
WARN: Unable to resolve link target: releases/about/validmind-community.qmd
WARN: Unable to resolve link target: releases/about/style-guide.qmd
WARN: Unable to resolve link target: guide/quickstart-try-developer-framework-with-jupyterhub.qmd

make

Updating Python documentation ...

Watching files for changes
Browse at http://localhost:5566/releases/2024-jul-31/release-notes.html
GET: /releases/2024-jul-31/release-notes.html
  /releases/about/overview.qmd (404: Not Found)
GET: /releases/2024-jul-31/release-notes.html
  /releases/about/style-guide.qmd (404: Not Found)
GET: /releases/2024-jul-31/release-notes.html
  /releases/about/validmind-community.qmd (404: Not Found)
GET: /releases/2024-jul-31/release-notes.html


GET: /releases/2024-jul-31/release-notes.html
